In [ ]:
import matplotlib.pyplot as plt
plt.style.use('seaborn-white')
import seaborn as sns
sns.set_style("white")

import scipy as sp
import numpy as np
import pandas as pd
import os,sys

In [ ]:
case = 'case2'
tmp = 'LT'
workdir = '/bigdata/greaneylab/wzhan097/proton_batterycase1-chg-modified/LT/'
data_path = os.path.join(workdir,'E_sol/H3O_OH',case)

In [ ]:
data_path

# Read structure

In [2]:
def readPOSCAR(filename, filedir = ''):
    with open(os.path.join(filedir,filename), 'r') as f:
        lines = f.readlines()
        
    title, unit , lines = lines[0],float(lines[1].split()[0]),lines[2:]
    box,eles,lines = lines[:3],lines[3:5],lines[5:]
    box = np.array([[float(e) for e in line.split()] for line in box])*unit
    eles = sum([[e]*int(num) for e, num in zip(eles[0].split(),eles[1].split())], [])
    
    corr = lines[0]
    def __coordinate(coor):
        if corr == 'Direct\n':
            return np.matmul(box, coor.T).T
        else:
            return coor
    
    coords = lines[1:1+len(eles)]
    coords = np.array([np.array([float(e) for e in line.split()]) for line in coords])
    coords = __coordinate(coords)
    atoms = {}
    for ele,coord in zip(eles,coords):
        atoms[ele] = atoms.get(ele,[])+[coord]
    for ele in atoms:
        atoms[ele] = np.array(atoms[ele])
    
    return box, unit, atoms
from numpy.linalg import inv
def writePOSCAR(box, unit, atoms, filename, filedir = ''):
    
    title = 'LiCl'
    
    lines = [title]
    lines.append('\t'+str(unit))
    
    boxline = ['\t\t'+'\t'.join(list(map(str,e))) for e in box]
    lines += boxline
    
    elename = ''
    elenum = ''
    coords = []
    
    
    def __coordinate(coor,corr='Direct'):
        if corr == 'Direct':
            return np.matmul(inv(box), coor.T).T
        else:
            return coor
    
    
    for key in atoms:
        elename+='\t'+key
        elenum+='\t'+str(len(atoms[key]))
        coord = __coordinate(atoms[key])
        coord = ['\t'+'\t'.join(list(map(str,e))) for e in coord]
        coords+=coord
        
    lines+=[elename,elenum]
    lines+=['Direct']
    lines+=coords
    
    with open(os.path.join(filedir,filename), 'w+') as f:
        f.writelines('\n'.join(lines))
    
    return 

# Change Structure

In [3]:
def findNearest(box, atom, aim_pool):
    """
    Find the nearest one to given atom from a group of candidate
    Input: 
        box: periodic box size
        atom: given atom (position, dimention=3d)
        aim pool: a list of positions.
    Return:
        index of closest atom in aim_pool
    
    """
    vecs = abs(np.array(aim_pool)-np.array(atom))
    for i in range(3):
        vecs[:,i] = np.minimum(vecs[:,i],box[i]-vecs[:,i])
    distance = np.linalg.norm(vecs,axis=1)
    return np.where(distance == min(distance))[0]
def distance(box, atom, aim_pool):
    vecs = abs(np.array(aim_pool, dtype='float')-np.array(atom, dtype='float'))
    for i in range(3):
        vecs[:,i] = np.minimum(vecs[:,i],box[i]-vecs[:,i])
    distance = np.linalg.norm(vecs,axis=1)
    return distance

In [4]:
def addProton(pos, atoms, box):
    min_dis = 1.8
    dis = min(distance([box[0,0],box[1,1],box[2,2]], pos, np.concatenate(list(atoms.values()))))
    if dis < min_dis:
        print('Not avaliable position')
        print(dis)
        return atoms
    else:
        atoms['H'] = np.concatenate((atoms['H'],[pos]))
        print('Action applied')
        return atoms
'''    
def addOH(pos, atoms, box):
    min_dis = 2
    if min(distance([box[0,0],box[1,1],box[2,2]], pos, np.concatenate(list(atoms.values())))) < min_dis:
        print('Not avaliable position')
        return atoms
    else:
        atoms['H'] = np.concatenate((atoms['H'],[pos]))
        print('Action applied')
        return atoms
'''

"    \ndef addOH(pos, atoms, box):\n    min_dis = 2\n    if min(distance([box[0,0],box[1,1],box[2,2]], pos, np.concatenate(list(atoms.values())))) < min_dis:\n        print('Not avaliable position')\n        return atoms\n    else:\n        atoms['H'] = np.concatenate((atoms['H'],[pos]))\n        print('Action applied')\n        return atoms\n"

In [ ]:
box, unit, atoms = readPOSCAR('POSCAR-LT', filedir = os.path.join(data_path,'relaxed_struct'))

In [ ]:
writePOSCAR(box, unit, atoms, 'test', filedir = '')

In [ ]:
np.concatenate((atoms['H'],[[1,2,3]]))

In [ ]:
np.concatenate(list(atoms.values())).shape

In [ ]:
LT_pos = (np.array([3.01159 ,13.32980 ,12.49876])+np.array([5.81096 ,12.22732  ,8.48433]))/2
LT_pos

In [ ]:
addProton(LT_pos, atoms, box)

In [ ]:
writePOSCAR(box, unit, atoms, 'POSCAR_extraProton', filedir = '')

In [ ]:
box, unit, atoms = readPOSCAR('POSCAR-RT', filedir = os.path.join(data_path,'relaxed_struct'))
RT_pos = (np.array([13.52940,9.22908,1.52398])+np.array([10.95449,11.32117,4.24529]))/2
#RT_pos = (RT_pos+np.array([12.83812,5.61727,3.45880]))/2
addProton(RT_pos, atoms, box)
writePOSCAR(box, unit, atoms, 'POSCAR_extraProton_RT', filedir = '')


In [ ]:
def fixAtom(file):
    with open(file,'r') as f:
        lines = f.readlines()
    for i in range(8,len(lines)):
        lines[i] = lines[i][:-1]+'\tF\tF\tF\n'
    
    with open(file,'w+') as f:
        f.writelines(lines)
    return lines

In [ ]:
os.listdir('/rhome/wzhan097/bigdata/proton_battery/E_sol/H3O_OH/case2-fix/')

In [ ]:
for dirname in os.listdir('/rhome/wzhan097/bigdata/proton_battery/E_sol/H3O_OH/case2-fix/'):
    if dirname[0] not in 'RL':
        continue
    print(dirname)
    fixAtom('/rhome/wzhan097/bigdata/proton_battery/E_sol/H3O_OH/case2-fix/'+dirname+'/POSCAR')

In [ ]:
def hydPos(O,H1,H2,box):
    O = np.array(O,dtype = float)
    H1 = np.array(H1,dtype = float)
    H2 = np.array(H2,dtype = float)
    
    def __periodic(vec,box):
        for i,e in enumerate(vec):
            while vec[i]>box[i]:
                vec[i] = vec[i]-box[i]
            while vec[i]<0:
                vec[i] = vec[i]+box[i]
        return vec
    
    def __diff(vec1,vec2,box):
        vec = np.zeros(3)
        for i,(e1,e2) in enumerate(zip(vec1,vec2)):
            if e2-e1>box[i]/2:
                vec[i] = e2-box[i]-e1
            elif e2-e1<-box[i]/2:
                vec[i] = e2+box[i]-e1
            else:
                vec[i] = e2-e1
        return vec
    
    v_H1 = __diff(O,H1,box)
    v_H2 = __diff(O,H2,box)
    _x = (v_H1+v_H2)/np.linalg.norm(v_H1+v_H2)
    _z = np.cross(v_H1,v_H2)
    _z = _z/np.linalg.norm(_z)
    _y = np.cross(_z,_x)
    trans = np.concatenate(([_x],[_y],[_z]),axis = 0).T
    Hyd=np.load(os.path.join('/rhome/wzhan097/shared/3x1x1PB-Weiyi/raptor/script','Hyd.npy'),allow_pickle=True)
    for i in range(len(Hyd)):
        _vec = np.array([Hyd[i,1:]],dtype = float).T
        _vec = np.matmul(trans,_vec).T
        Hyd[i,1:] = _vec+O
        Hyd[i,1:] = __periodic(Hyd[i,1:],box)
    return Hyd

In [ ]:
box, unit, atoms

In [ ]:
waterC1 = {}
waterC1['O'] = atoms['O'][[24,47,78]]
waterC1['H'] = atoms['H'][[48,49,94,95,156,157],]

In [ ]:
waterC2 = {}
waterC2['O'] = atoms['O'][[47,78]]
waterC2['H'] = atoms['H'][[94,95,156,157],]

In [ ]:
for e in waterC1:
    waterC1[e] = waterC1[e]-[9.5,3,1]+[5,5,5]
waterC1

In [ ]:
for e in waterC2:
    waterC2[e] = waterC2[e]-[9.5,3,1]+[20,5,5]
waterC2

In [ ]:
newA = {}
newA['O'] = np.concatenate((waterC1['O'],waterC2['O']))
newA['H'] = np.concatenate((waterC1['H'],waterC2['H']))

In [ ]:
newA

In [ ]:
hydPos(newA['O'][4],newA['H'][8],newA['H'][9],[newBox[0,0],newBox[1,1],newBox[2,2]])

In [ ]:
newA['H'][1] = [22.545076591787666, 7.811578788676513, 8.839151987117049]
newA['H'][8] = [21.959667103885376, 6.4730969554054845, 8.038488579789208]
newA['H'][9] = [23.362973394927664, 7.136848325638254, 7.573605705982493]

In [ ]:
writePOSCAR(newBox, unit, newA, 'POSCAR_LT_1', filedir = '')

In [ ]:
newStruct = {}
newStruct['Li'] = atoms['Li'][8:9]

newStruct['O'] = atoms['O'][[36,90,82,80]]
newStruct['H'] = atoms['H'][[72,73,132,181,160,161,164,165],]

In [ ]:
newStructClH2O = {}
newStructClH2O['Cl'] = atoms['Cl'][2:3]

newStructClH2O['O'] = atoms['O'][72:73]
newStructClH2O['H'] = atoms['H'][[144,145],]

In [ ]:
for e in newStruct:
    newStruct[e] = newStruct[e]-[1.5,5,4]+[5,5,5]

In [ ]:
newStruct

In [ ]:
newStruct

In [ ]:
newStructClH2O

In [ ]:
for e in newStructClH2O:
    newStructClH2O[e] = newStructClH2O[e]-[1,0,8] + [15,5,5]

In [ ]:
newStructClH2O

In [ ]:
newA = {}
newA['Li'] = newStruct['Li']
newA['Cl'] = newStructClH2O['Cl']
newA['O'] = np.concatenate((newStruct['O'],newStructClH2O['O']))
newA['H'] = np.concatenate((newStruct['H'],newStructClH2O['H']))


In [ ]:
newA

In [ ]:
newBox = np.array([[30,  0. ,  0. ],
        [ 0. , 20,  0. ],
        [ 0. ,  0. , 20]])

In [ ]:
writePOSCAR(newBox, unit, newA, 'POSCAR_LT_1', filedir = '')

In [ ]:
newA['H'][7] = [25.293580290007903, 12.252061644556422, 9.06274635912817]
newA['H'][8] = [26.533003682429996, 12.698729190394094, 10.082362726282344]
newA['H'][9] = [25.006017414541848, 12.70271995941247, 10.624808599865318]

In [ ]:
newA['O'][3],newA['H'][4],newA['H'][5]

In [ ]:
hydPos(newA['O'][4],newA['H'][8],newA['H'][9],[newBox[0,0],newBox[1,1],newBox[2,2]])

In [ ]:
newA['Cl'] = np.concatenate((newA['Cl'],[newA['Li'][0]+chg/2.08375 * 2.31631]))

In [ ]:
#newA['O'][0]+(newA['O'][0]-newA['Li'][0])
chg = (newA['O'][0]-newA['Li'][0])

In [ ]:
chg/2.08375 * 2.31631

In [ ]:
newA['O'] = newA['O'][1:]
newA['H'] = newA['H'][2:]

In [ ]:
newA

In [19]:
dataPath = '/rhome/wzhan097/bigdata/proton_battery/E_sol/H3O_OH/case1-chg-modified/RT/'

In [20]:
box, unit, atoms = readPOSCAR('POSCAR', filedir = dataPath)

In [21]:
atoms

{'Li': array([[ 1.13823873,  6.56858223,  0.46029506],
        [12.25633316,  4.86507093,  9.49621898],
        [ 1.01186823,  7.58722988,  8.78492585],
        [ 7.80367061, 11.31053967,  4.7062264 ],
        [10.56241278, 10.27470801, 11.604006  ],
        [ 7.84497169,  7.64402475, 14.20167575],
        [ 9.8227187 ,  2.52788997,  0.87180741],
        [ 5.0245315 ,  3.84814179,  2.47997725],
        [12.44669829, 13.93900108,  1.77651671],
        [ 6.60142681, 11.55739259, 10.23761338],
        [ 0.54617778, 11.89293198,  6.72266387],
        [11.89935678,  6.57446898,  4.32740566],
        [ 1.99202059, 14.03443078, 13.70276598]]),
 'Cl': array([[ 0.69921719,  7.97480641, 11.17096887],
        [ 3.12253063,  8.01588453,  0.84048674],
        [ 1.87327635,  6.72551452,  4.5330957 ],
        [ 8.70750356, 10.10597962,  6.7843329 ],
        [ 1.23373234,  5.17331301, 13.13820948],
        [11.21908086,  5.44917488, 12.98720902],
        [ 4.84110663, 13.06198075,  2.28039955],
      

In [22]:
(np.array([0.77373,   0.37069,   0.89567])+np.array([0.79195,   0.75577,   0.89525]))/2

array([0.78284, 0.56323, 0.89546])